In [234]:
import pandas as pd
import numpy as np
import sys

localized = pd.read_table('/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/unmapped/localized_start_with_Y.bed', header=None, nrows=10000)
localized_end = pd.read_table('/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/unmapped/localized_end_with_Y.bed', header=None, nrows=10000)

localized.columns = ['chrom', 'start', 'NULL', 'idx']
localized.drop('NULL', axis=1, inplace=True)
localized['end'] = localized_end[1]
localized = localized[localized.chrom!='chr24']
localized['chrom'] = localized['chrom'].replace('chr23', 'chrX')

localized[['chrom', 'start', 'end', 'idx']].to_csv(
    '/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/unmapped/localized_regions_hg38_for_conversion.bed',
    header=None, sep='\t', index=None)
print(len(localized))

9643


In [240]:
%%bash
cd /home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/unmapped/
CHAIN=$MY_HOME/alt_haplotypes/intermediate_files/t2t_comparison/grch38.t2t-chm13-v1.1.over.chain.gz

/oak/stanford/groups/dpwall/computeEnvironments/liftOver -ends=100000 localized_regions_hg38_for_conversion.bed $CHAIN localized_regions_T2T.bed localized_regions_T2T_liftover_fail.bed
wc -l localized_regions_T2T_liftover_fail.bed

3902 localized_regions_T2T_liftover_fail.bed


Reading liftover chains
Mapping coordinates


In [238]:
a = pd.read_table('/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/unmapped/localized_regions_T2T.bed', header=None)
a

,0,1,2,3
0,chr12,7684933,7686826,10
1,chr8,31556483,31557290,17
2,chr9,39509028,78954909,21
3,chr11,113592231,114154096,41
4,chr17,18608655,18900148,52
...,...,...,...,...
7687,chr4,8131854,14268089,56206
7688,chr11,98841265,102519584,56209
7689,chr17,80408435,80784089,56213
7690,chr9,40018535,41976205,56217


In [212]:
import pandas as pd
import numpy as np

a = pd.read_table('/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/unmapped/localized_regions_T2T.bed', header=None)
a.columns = ['chrom', 'start', 'end', 'read_idx', 'alignment_idx']
a['length'] = a.end-a.start
#a.drop('alignment_idx', axis=1, inplace=True)
a = a.groupby(['read_idx', 'chrom']).aggregate(
    {'start': lambda x: (min(x), max(x)),
     'end': lambda x: (min(x), max(x)),
    'length': sum,
    'alignment_idx': lambda x:min(x) })
a = a.reset_index()
a = a.sort_values(['read_idx', 'length']) ### CHANGE TO ORDER BY LENGTH< ALINGMENT_IDX
a = a[~a['read_idx'].duplicated()] ## CHANGE TO KEEP FIRST DUP
a.index = a.read_idx
a['start_true'] = np.min([a.start.apply(min).values, a.end.apply(min).values], axis=0)
a['end_true'] = np.max([a.start.apply(max).values, a.end.apply(max).values], axis=0)
a.drop(['start', 'end', 'read_idx', 'length', 'alignment_idx'], axis=1, inplace=True)
print(len(a))
a.to_csv('/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/unmapped/localized_regions_T2T_good_ids.bed', header=None, sep='\t')

91061


In [214]:
print(np.median(a.start_true-a.end_true))

-688773.0


In [179]:
len(a)

66583

In [132]:
a

,chrom,start_true,end_true
read_idx,,,
0,chr6,32412424,32412472
10,chr12,7684933,7686826
17,chr8,31556483,31557290
21,chr9,39363294,79943610
35,chr9,39363294,77052112
...,...,...,...
56209,chr11,98841265,102519584
56210,chr9,39363294,41738267
56213,chr17,80408435,80784089


In [122]:
a

,chrom,start_true,end_true
read_idx,,,
0,chr6,32412424,32412472
10,chr12,7684933,7686826
17,chr8,31556483,31557290
21,chr9,39363294,79943610
35,chr9,39363294,77052112
...,...,...,...
56209,chr11,98841265,102519584
56210,chr9,39363294,41738267
56213,chr17,80408435,80784089


In [101]:
a = a.sort_values(['read_idx', 'counter'])
a = a[~a['read_idx'].duplicated()]

In [75]:
a = a[a.chrom!='SPLIT']
a['start_true'] = np.min([a.start.apply(min).values, a.end.apply(min).values], axis=0)
a['end_true'] = np.max([a.start.apply(max).values, a.end.apply(max).values], axis=0)
a.drop(['start', 'end'], axis=1, inplace=True)
a.to_csv('/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/unmapped/localized_regions_T2T_good_ids.bed', header=None, sep='\t')

In [76]:
print(len(a))

4721


In [77]:
Counter(a.chrom).most_common(10)

[('chr6', 524),
 ('chr9', 463),
 ('chr14', 339),
 ('chr12', 338),
 ('chr1', 287),
 ('chr3', 242),
 ('chr2', 236),
 ('chr15', 226),
 ('chr4', 199),
 ('chr5', 186)]

In [78]:
print(np.median(a.end_true-a.start_true))
print(len(a))

49927.0
4721


In [40]:
import pandas as pd
import numpy as np
import sys

batch_number = 0 #int(sys.argv[1])
localized = pd.read_table('/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/unmapped/localized_%04d.tsv' % batch_number, comment='#', header=None)
localized.index = 100000*batch_number+localized.index
localized = localized[~np.isnan(localized[6])][[6,7,8]]
localized['chrom'] = [('chr%i'% int(i)).replace('23', 'X') for i in localized[6]]
localized['start'] = [int(i) for i in localized[7]]
localized['end'] = [int(i) for i in localized[8]]
localized = localized[localized.chrom!='chr24']
localized['index'] = localized.index
localized[['chrom', 'start', 'start', 'index']].to_csv('/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/unmapped/localized_%04d_start.bed' % batch_number,
                                                     header=None, index=None, sep='\t')
localized[['chrom', 'end', 'end', 'index']].to_csv('/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/unmapped/localized_%04d_end.bed' % batch_number,
                                                     header=None, index=None, sep='\t')
print('saved to /home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/unmapped/localized_%04d.bed' % batch_number)

saved to /home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/unmapped/localized_0000.bed


In [41]:
%%bash
head /home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/unmapped/localized_0000_start.bed

chr6	32518741	32518741	0
chr12	7670646	7670646	10
chr8	31275335	31275335	17
chr9	39495523	39495523	21
chr9	40571113	40571113	35
chr6	31326471	31326471	39
chr11	113581871	113581871	41
chr17	18661830	18661830	52
chr15	65384279	65384279	58
chr5	43089925	43089925	60


In [43]:
%%bash

CHAIN=$MY_HOME/alt_haplotypes/intermediate_files/t2t_comparison/grch38.t2t-chm13-v1.1.over.chain.gz


BED_IN=/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/unmapped/localized_0000_start.bed
BED_OUT=/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/unmapped/localized_0000_T2T_start.bed
BED_UNMAPPED=/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/unmapped/localized_0000_start_unmapped.bed

/oak/stanford/groups/dpwall/computeEnvironments/liftOver $BED_IN $CHAIN $BED_OUT $BED_UNMAPPED


BED_IN=/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/unmapped/localized_0000_end.bed
BED_OUT=/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/unmapped/localized_0000_T2T_end.bed
BED_UNMAPPED=/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/unmapped/localized_0000_end_unmapped.bed

/oak/stanford/groups/dpwall/computeEnvironments/liftOver $BED_IN $CHAIN $BED_OUT $BED_UNMAPPED

#\rm $BED_IN
#\rm $BED_UNMAPPED


Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates


In [44]:
import pysam

In [39]:
%%bash
head /home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/unmapped/localized_0000_end_unmapped.bed

#Split in new
chr6	32518741	32558449	0
#Split in new
chr9	39495523	67182501	21
#Split in new
chr9	40571113	68356015	35
#Split in new
chr5	43089925	53917542	60
#Split in new
chr1	121422747	145905397	138
